In [3]:
import gensim
import xgboost as xgb

from capstone_project import preprocessor as pre

In [4]:
train_data = pre.load_data()
y = train_data["is_duplicate"].values
model_directory = "../output/models/"
skf = pre.load_pickle(model_directory, "kfolds.pkl") 

In [5]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("../data/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [ ]:
def modelfit(alg, x_train, x_test, y_train, y_test, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    """https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/"""
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(x_train, label=y_train)
        #xgtest = xgb.DMatrix(x_test)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='logloss', early_stopping_rounds=early_stopping_rounds, stratified=True)
        print("\nBest number of iterations: {}\n".format(cvresult.shape[0]))
        print(cvresult[-10:])
        alg.set_params(n_estimators=cvresult.shape[0])

    # Fit the algorithm on the data
    alg.fit(x_train, y_train, eval_metric='logloss')

    # Predict training set:
    train_predictions = alg.predict(x_train)
    train_predprob = alg.predict_proba(x_train)[:, 1]

    # Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train, train_predictions))
    print("Logloss (Train): %f" % metrics.log_loss(y_train, train_predprob))

    #     Predict on testing data:
    test_pred = alg.predict_proba(x_test)[:, 1]
    print('Logloss (Test): %f' % metrics.log_loss(y_test, test_pred))

    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    #plt.show()


In [ ]:
word_features = pre.FeatureTransformer()
word2vec_transform = pre.Word2vecTransformer()
word2vec_features = pre.VectorFeatureTransformer()
mms = MinMaxScaler()
clf = xgboost()